In [1]:
import os, sys
import pandas as pd
import numpy as np
import cv2
import shutil

In [2]:
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(PATH)
from utilities.sqlcontroller import SqlController
from utilities.file_location import FileLocationManager

from utilities.utilities_cvat_neuroglancer import get_structure_number, NumpyToNeuroglancer, \
get_segment_properties, get_db_structure_infos

Connecting dklab@192.168.1.12:3306


In [3]:
DATA_PATH = '/net/birdstore/Active_Atlas_Data/data_root'
csvfile = os.path.join(DATA_PATH, 'atlas_data', 'DK39.PM.Nucleus.csv')
DK39_df = pd.read_csv(csvfile, dtype={'Layer':'str', 'X':np.uint16, 'Y':np.uint16,'Section':np.uint16})
DK39_df = DK39_df.loc[DK39_df['Layer'] == 'premotor']
csvfile = os.path.join(DATA_PATH, 'atlas_data', 'DK52.PM.Nucleus.csv')
DK52_df = pd.read_csv(csvfile, dtype={'Layer':'str', 'X':np.uint16, 'Y':np.uint16,'Section':np.uint16})
DK52_df = DK52_df.loc[DK52_df['Layer'] == 'PM nucleus']
animal = 'DK52'
sqlController = SqlController(animal)
fileLocationManager = FileLocationManager(animal)
OUTPUT_DIR = os.path.join(fileLocationManager.neuroglancer_data, 'points')
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [4]:
resolution = sqlController.scan_run.resolution
SCALE = (10 / resolution)
print(SCALE)

aligned_shape = np.array((sqlController.scan_run.height,sqlController.scan_run.width))
print('aligned shape', aligned_shape)
downsampled_aligned_shape = np.round(aligned_shape // SCALE).astype(int)
col_length = int(round(sqlController.scan_run.width / SCALE))
row_length = int(round(sqlController.scan_run.height / SCALE))
z_length = sqlController.get_section_count(animal)
container_volume = np.zeros((row_length, col_length, z_length), dtype=np.uint8)
print('Shape of container volume', container_volume.shape, container_volume.dtype)

30.769230769230766
aligned shape [36000 65000]
Shape of container volume (1170, 2112, 486) uint8


In [5]:
db_structure_infos = get_db_structure_infos()
[(number, f'{structure}: {label}') for structure, (label, number, sided) in db_structure_infos.items()]


[(3, '3N: Oculomotor nucleus'),
 (4, '4N: Trochlear nucleus'),
 (5, '5N: Trigeminal motor nucleus'),
 (6, '6N: Abducens nucleus'),
 (7, '7N: Facial nucleus'),
 (9, '7n: Facial nerve'),
 (1, '10N: Dorsal nucleus of vagus nerve'),
 (2, '12N: Hypoglossal nucleus'),
 (8, 'Amb: Nucleus ambiggus'),
 (28, 'AP: Area postrema'),
 (10, 'DC: Dorsal cochlea nucleus'),
 (13, 'LRt: Lateral reticular nucleus'),
 (12, 'LC: Locus corelus'),
 (11, 'IC: Inferior colliculus'),
 (25, 'VCA: Ventral cochlea nucleus, anterior'),
 (26, 'VCP: Ventral cochlea nucleus, posterior'),
 (27, 'VLL: Ventral lateral lemniscus'),
 (14, 'PBG: Parabigeminal nucleus'),
 (15, 'Pn: Pontine grey'),
 (16, 'R: Red nucleus'),
 (17, 'RtTg: Reticulotegmental nucleus'),
 (18, 'SC: Superior colliculus'),
 (21, 'Sp5C: Spinal-trigeminal nucleus, caudalis'),
 (22, 'Sp5I: Spinal-trigeminal nucleus, interpolaris'),
 (23, 'Sp5O: Spinal-trigeminal nucleus, oralis'),
 (20, 'SNR: Substantia niagra, reticular'),
 (19, 'SNC: Substantia niagra, 

In [6]:
point_size = 50
DK52_single_point = np.zeros((point_size, point_size), dtype=np.uint8) + 180

In [7]:
DK52_single_point

array([[180, 180, 180, ..., 180, 180, 180],
       [180, 180, 180, ..., 180, 180, 180],
       [180, 180, 180, ..., 180, 180, 180],
       ...,
       [180, 180, 180, ..., 180, 180, 180],
       [180, 180, 180, ..., 180, 180, 180],
       [180, 180, 180, ..., 180, 180, 180]], dtype=uint8)

In [8]:
debug = True
shift_point = DK52_single_point.shape[0]//2
for index, row in DK52_df.iterrows():
    x = row['X'] //SCALE
    y = row['Y']//SCALE
    z = row['Section']
    
    col_start = int(round(x))
    row_start = int(round(y))
    z_start = int(round(z))
    row_end = row_start + DK52_single_point.shape[0]
    col_end = col_start + DK52_single_point.shape[1]
    z_end = int(z_start +  0)
    if debug:
        print('Y', str(row_start).rjust(4),  str(row_end).rjust(4),
              'X',  str(col_start).rjust(4), str(col_end).rjust(4),
              'Z',  str(z_start).rjust(4), str(z_end).rjust(4),
              end="\n")
    
    try:
        container_volume[row_start:row_end, col_start:col_end, z_start] += DK52_single_point
    except ValueError as ve:
        print(ve)



Y  418  468 X 1255 1305 Z  125  125
Y  346  396 X 1272 1322 Z  125  125
Y  349  399 X 1306 1356 Z  125  125
Y  343  393 X 1310 1360 Z  125  125
Y  352  402 X 1321 1371 Z  125  125
Y  348  398 X 1357 1407 Z  125  125
Y  417  467 X 1377 1427 Z  125  125
Y  345  395 X 1410 1460 Z  125  125
Y  407  457 X 1426 1476 Z  125  125
Y  525  575 X 1434 1484 Z  125  125
Y  526  576 X 1435 1485 Z  125  125
Y  620  670 X 1251 1301 Z  133  133
Y  649  699 X 1259 1309 Z  133  133
Y  653  703 X 1260 1310 Z  133  133
Y  619  669 X 1270 1320 Z  133  133
Y  633  683 X 1286 1336 Z  133  133
Y  640  690 X 1289 1339 Z  133  133
Y  646  696 X 1262 1312 Z  137  137
Y  649  699 X 1265 1315 Z  137  137
Y  619  669 X 1256 1306 Z  141  141
Y  635  685 X 1261 1311 Z  141  141
Y  632  682 X 1266 1316 Z  141  141
Y  599  649 X 1274 1324 Z  141  141
Y  587  637 X 1275 1325 Z  141  141
Y  651  701 X 1295 1345 Z  141  141
Y  605  655 X 1255 1305 Z  145  145
Y  643  693 X 1270 1320 Z  145  145
Y  593  643 X 1281 1331 Z  1

In [9]:
np.mean(container_volume)

0.021144226905543777

In [10]:
ng = NumpyToNeuroglancer(container_volume, [10000, 10000, 20000])
ng.preview()
ng.init_precomputed(OUTPUT_DIR)
ng.add_segment_properties(get_segment_properties())
ng.add_downsampled_volumes()
ng.add_segmentation_mesh()

A new layer named segmentation_[10000, 10000, 20000] is added to:
http://127.0.0.1:41537/v/bee846bed79b40ccd389549fbd4cfe4ed264af2a/


Downloading:  27%|██▋       | 163/608 [00:00<00:00, 1324.00it/s]

Volume Bounds:  Bbox([0, 0, 0],[1170, 2112, 486], dtype=int32)
Selected ROI:   Bbox([0, 0, 0],[1170, 2112, 486], dtype=int32)



Downloading: 100%|██████████| 608/608 [00:01<00:00, 428.91it/s] 

Uploading: 100%|██████████| 160/160 [00:00<00:00, 517.02it/s]

Uploading:   0%|          | 0/40 [00:00<?, ?it/s]

Uploading:  98%|█████████▊| 39/40 [00:00<00:00, 277.40it/s]

Uploading: 100%|██████████| 40/40 [00:00<00:00, 277.01it/s]


Uploading:   0%|          | 0/12 [00:00<?, ?it/s]



Uploading:   0%|          | 0/4 [00:00<?, ?it/s]



Tasks:   6%|▋         | 1/16 [00:02<00:34,  2.31s/it]/s]




Uploading: 100%|██████████| 4/4 [00:00<00:00, 18.89it/s]



Uploading:   0%|          | 0/10 [00:00<?, ?it/s]



Uploading:   0%|          | 0/5 [00:00<?, ?it/s]





Uploading:   0%|          | 0/3 [00:00<?, ?it/s]






Tasks:  12%|█▎        | 2/16 [00:03<00:25,  1.83s/it]A







Uploading: 100%|██████████| 2/2 [00:00<00:00,  9.60it/s]








Downloading:  24%|██▎       | 143/608 [00:00<00:00, 1386.08it/s]







Downloading:  26%|██▌       | 159/608 [00:00<00:00, 1381.18it/s]







Downloading:  29%|██▉       | 175/608

Downloading:  25%|██▍       | 151/608 [00:00<00:00, 1255.88it/s]











Downloading:  29%|██▊       | 174/608 [00:00<00:00, 1255.88it/s]











Downloading:  31%|███       | 188/608 [00:00<00:00, 1255.88it/s]











Downloading:  32%|███▏      | 196/608 [00:00<00:00, 1255.88it/s]











Downloading:  35%|███▍      | 210/608 [00:00<00:00, 1255.88it/s]











Downloading:  37%|███▋      | 225/608 [00:00<00:00, 1255.88it/s]











Downloading:  39%|███▊      | 235/608 [00:00<00:00, 1255.88it/s]











Downloading:  43%|████▎     | 262/608 [00:00<00:00, 1255.88it/s]











Downloading:  49%|████▉     | 297/608 [00:00<00:00, 1255.88it/s]











Downloading:  55%|█████▍    | 332/608 [00:00<00:00, 1255.88it/s]











Downloading:  62%|██████▏   | 374/608 [00:00<00:00, 1255.88it/s]











Downloading:  68%|██████▊   | 411/608 [00:00<00:00, 1255.88it/s]











Downloading:  73%|███████▎  | 446/608 [00:00<00:00, 1255.88it/s]











Downloading:

...

Uploading: 100%|██████████| 160/160 [00:00<00:00, 922.03it/s]





Uploading:   0%|          | 0/40 [00:00<?, ?it/s]






Uploading:   0%|          | 0/12 [00:00<?, ?it/s]







Tasks:  94%|█████████▍| 15/16 [00:21<00:01,  1.38s/it]









Uploading: 100%|██████████| 4/4 [00:00<00:00, 17.05it/s]

Uploading:   0%|          | 0/10 [00:00<?, ?it/s]

Uploading:   0%|          | 0/5 [00:00<?, ?it/s]


Uploading:   0%|          | 0/3 [00:00<?, ?it/s]



Tasks:   0%|          | 0/30 [00:00<?, ?it/s]




Uploading: 100%|██████████| 2/2 [00:00<00:00, 10.15it/s]





Downloading:  32%|███▏      | 164/512 [00:00<00:00, 1505.34it/s]




Downloading:  33%|███▎      | 167/512 [00:00<00:00, 1517.13it/s]




Downloading:  36%|███▌      | 182/512 [00:00<00:00, 1517.13it/s]




Downloading:  36%|███▌      | 185/512 [00:00<00:00, 1517.13it/s]




Downloading:  37%|███▋      | 191/512 [00:00<00:00, 1517.13it/s]




Downloading:  38%|███▊      | 194/512 [00:00<00:00, 1517.13it/s]




Downloading:  40%|

Tasks:  17%|█▋        | 5/30 [00:02<00:11,  2.25it/s]567.20it/s]

Downloading:   0%|          | 0/320 [00:00<?, ?it/s]

Downloading:  64%|██████▍   | 205/320 [00:00<00:00, 1896.19it/s]

Downloading:  65%|██████▌   | 208/320 [00:00<00:00, 1897.01it/s]

Downloading:  66%|██████▋   | 212/320 [00:00<00:00, 1897.01it/s]

Downloading:  67%|██████▋   | 215/320 [00:00<00:00, 1897.01it/s]

Downloading:  71%|███████▏  | 228/320 [00:00<00:00, 1897.01it/s]

Downloading:  73%|███████▎  | 234/320 [00:00<00:00, 1897.01it/s]

Downloading:  76%|███████▋  | 244/320 [00:00<00:00, 1897.01it/s]

Downloading:  80%|████████  | 256/320 [00:00<00:00, 1897.01it/s]

Downloading:  89%|████████▉ | 285/320 [00:00<00:00, 1897.01it/s]

Downloading:  95%|█████████▌| 304/320 [00:00<00:00, 1897.01it/s]

Downloading: 100%|██████████| 320/320 [00:00<00:00, 1897.01it/s]

Downloading: 100%|██████████| 320/320 [00:00<00:00, 1897.01it/s]

Downloading: 100%|██████████| 320/320 [00:00<00:00, 1897.01it/s]

Downloading: 100%|████

Downloading:  45%|████▍     | 230/512 [00:00<00:00, 1564.34it/s]

Downloading:  51%|█████     | 260/512 [00:00<00:00, 1564.34it/s]

Downloading:  55%|█████▌    | 283/512 [00:00<00:00, 1564.34it/s]

Downloading:  59%|█████▉    | 302/512 [00:00<00:00, 1564.34it/s]

Downloading:  62%|██████▏   | 318/512 [00:00<00:00, 1564.34it/s]

Downloading:  65%|██████▌   | 334/512 [00:00<00:00, 1564.34it/s]

Downloading:  69%|██████▉   | 355/512 [00:00<00:00, 1564.34it/s]

Downloading:  76%|███████▌  | 387/512 [00:00<00:00, 1564.34it/s]

Downloading:  81%|████████  | 414/512 [00:00<00:00, 1564.34it/s]

Downloading:  89%|████████▉ | 455/512 [00:00<00:00, 1564.34it/s]

Downloading: 100%|█████████▉| 511/512 [00:00<00:00, 1564.34it/s]

Downloading: 100%|██████████| 512/512 [00:00<00:00, 1564.34it/s]

Downloading: 100%|██████████| 512/512 [00:00<00:00, 1564.34it/s]




Tasks:  33%|███▎      | 10/30 [00:37<02:08,  6.44s/it]77.01it/s]



Downloading:   0%|          | 0/512 [00:00<?, ?it/s]



Downloading:  3

Downloading:   0%|          | 0/40 [00:00<?, ?it/s]








Downloading: 100%|██████████| 40/40 [00:00<00:00, 325.07it/s]








Downloading: 100%|██████████| 40/40 [00:00<00:00, 325.07it/s]








Downloading: 100%|██████████| 40/40 [00:00<00:00, 325.07it/s]








Downloading: 100%|██████████| 40/40 [00:00<00:00, 325.07it/s]








Downloading: 100%|██████████| 40/40 [00:00<00:00, 325.07it/s]








Downloading: 100%|██████████| 40/40 [00:00<00:00, 325.07it/s]








Downloading: 100%|██████████| 40/40 [00:00<00:00, 325.07it/s]








Downloading: 100%|██████████| 40/40 [00:00<00:00, 325.07it/s]








Tasks:  60%|██████    | 18/30 [00:50<00:09,  1.23it/s]07it/s]










Tasks:  63%|██████▎   | 19/30 [00:50<00:06,  1.65it/s]











Downloading:   0%|          | 0/64 [00:00<?, ?it/s]











Downloading:  66%|██████▌   | 42/64 [00:00<00:00, 309.41it/s]











Downloading:  78%|███████▊  | 50/64 [00:00<00:00, 343.11it/s]











Downloading:  84%|████████▍ | 54/

Downloading: 100%|██████████| 40/40 [00:00<00:00, 329.76it/s]








Downloading: 100%|██████████| 40/40 [00:00<00:00, 329.76it/s]








Downloading: 100%|██████████| 40/40 [00:00<00:00, 329.76it/s]








Downloading: 100%|██████████| 40/40 [00:00<00:00, 329.76it/s]








Downloading: 100%|██████████| 40/40 [00:00<00:00, 329.76it/s]








Downloading: 100%|██████████| 40/40 [00:00<00:00, 329.76it/s]








Tasks:  93%|█████████▎| 28/30 [00:52<00:00,  5.23it/s]76it/s]










Tasks:  97%|█████████▋| 29/30 [00:52<00:00,  5.86it/s]











Tasks:  76%|███████▌  | 760/999 [00:15<00:04, 53.70it/s]





Tasks: 100%|██████████| 999/999 [00:20<00:00, 49.92it/s].74it/s]
